In [1]:
from skimage.morphology import binary_closing, binary_opening, erosion
import numpy as np

In [2]:
selem = np.zeros((25, 25))

ci,cj=12, 12
cr=13

# Create index arrays to z
I,J=np.meshgrid(np.arange(selem.shape[0]),np.arange(selem.shape[1]))

# calculate distance of all points to centre
dist=np.sqrt((I-ci)**2+(J-cj)**2)

# Assign value of 1 to those points where dist<cr:
selem[np.where(dist<=cr)]=1

In [3]:
%matplotlib inline
import numpy as np
from scipy import ndimage
from PIL import Image
import os

In [8]:
def bg_removal(input_path, output_path): 
    """
    remove background from image using erosion 
    Args:
        input_path: input image folder path
        output_path: path to save the background removed image
    Returns:
        background removed image(s) 
    """
    os_path = os.listdir(input_path)
    for img_name in os_path[:]:
        bg_frac = 0
        thres = 220
        img = Image.open(input_path + img_name)
        im_arr = np.array(img)

        R = im_arr[:, :, 0]
        G = im_arr[:, :, 1]
        B = im_arr[:, :, 2]
        while bg_frac < 0.2: 
            bg_mask = ((R>thres) | (B>thres))# & (G < 100)
            bg_frac = bg_mask.sum()/len(bg_mask.flatten())
            thres -= 5
        # we use opening first since our mask is reversed (the foreground and background are reversed here)
        bg_mask = binary_closing(erosion(binary_opening(bg_mask, selem), np.ones((3, 3))), np.ones((5,5)))

        #Get biggest blob
        label, num_label = ndimage.label(~bg_mask)
        size = np.bincount(label.ravel())
        biggest_label = size[1:].argmax() + 1
        bg_mask = label == biggest_label
        
        im_arr[~bg_mask, 0] = 0
        im_arr[~bg_mask, 1] = 0
        im_arr[~bg_mask, 2] = 0
        
        img = Image.fromarray(im_arr)
        img.save(output_path + img_name)
  
   

In [10]:
bg_removal("rld_img_cb/bacterial_leaf_blight/", "bgr_rmv_erosion/bacterial_leaf_blight/")
bg_removal("rld_img_cb/blast/", "bgr_rmv_erosion/blast/")
bg_removal("rld_img_cb/brown_spot/", "bgr_rmv_erosion/brown_spot/")
bg_removal("rld_img_cb/healthy/", "bgr_rmv_erosion/healthy/")
bg_removal("rld_img_cb/leaf_smut/", "bgr_rmv_erosion/leaf_smut/")
bg_removal("rld_img_cb/tungro/", "bgr_rmv_erosion/tungro/")